<a href="https://colab.research.google.com/github/aeholbrook/python_csv_writeup/blob/main/python_csv_writeup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A Write-up on CSV Imports and List Manipulation


##Intro

Since we talked today about the basics of csv imports and working with datasets, I thought I'd put together a brief notebook detailing some different ways to work with lists and how they affect performance. 


###Python stuff

This notebook introduces a few essential python tools, including the `time` library, responsible for reading the system clock and measuring the passage of time. We'll be using it to compare the efficiency of different pieces of code. 

I'm also going to briefly cover the python library `pandas`, which will be discussed in the next lesson, and show you how to do this same assignment in that context. I think you'll see why `pandas` is such a useful tool.

###R Stuff

Finally, I'm going to show you how to do this same assignment in another popular data science language, `R`. I think that even though `python` is a great primary language and will likely soon become *lingua franca* in the Data Science community, there are still a lot of folks using `R`, especially those working in research and science. I used it a quite often when I was a neuroscience researcher. 


In introducing `R`, I'll show you one of the coolest things that `notebooks` can do, which is called `%line magic%`. Put simply, the specific flavor we'll be using, `rmagic`, allows you to switch seamlessly between `R` code and `python` code directly in the notebook. There are a lot of other uses for this, but here we'll just stick with this one application. 


###Why You Should Read This

My hope with this write-up is that you'll see that:

*   There are a lot of ways to solve the same problem with code. Just because someone else does it differently doesn't mean you did it wrong.
*   Code can be used in a litany of ways, but sometimes writing code to emphasize performance and 'prettiness' can be detrimental to the readability of your code. Fancy $\neq$ readable!
*   The same skills you're developing now in `python` are just as easily applied to other languages like `R` with much less effort than it took to learn your first language. When you see a job posting that calls for a language like `R`, `SAS`, or `MatLab`, don't fret! By the end of this course you'll be closer to proficient in these languages than you think!




---



##Loading Extensions and Libraries

Let's start by loading the `rpy2` extension. You don't really need to know much about this, other than that it's responsible for allowing `R` and `python` to interact with each other later on. It's generally good practice to start off by loading & importing all the dependencies you're going to need for your code later on. Also, if you've got a lot of code and you're importing as needed, you might import the same thing a few times, or reimport it under a different name. This can make your code run slower or not at all, and can make things confusing to others reading what you've made!

We're also going to be importing the following `python` libraries:

*   `os`, which is responsible for interacting with your computer's operating system. This is the library that'll let us `open` and `read` files.

*   `csv`, which is what allows `python` to read and interpret `.csv`, or `comma-separated values` files. Our `cereal.csv` file is one of these.

*   `time`, which introduces tools responsible for reading the system clock and measuring the passage of time.

In [ ]:
%load_ext rpy2.ipython
!git clone http://github.com/aeholbrook/python_csv_writeup/
!cd python_csv_writeup

#for finding and reading our csv file
import os
import csv
#for getting the system clock to measure performance
import time
#for dataset manipulation
import pandas as pd

Cloning into 'python_csv_writeup'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 44 (delta 7), reused 37 (delta 5), pack-reused 0
Unpacking objects: 100% (44/44), done.


##Base `python` + for-loops

First, let's go over the solution we discussed today in class. I will explain this line-by-line here, which may serve as a review for what we covered in class. Let's get started! 

```
cereal_csv = os.path.join("Resources","cereal.csv")
```
This creates a string, `cereal.csv`, which contains a `path` for `python` to take to find the `cereal.csv` file, which contains our cereal list to be processed. It's a little different from the one we covered, because the file structure in this notebook is a little different than the directory structure from our assignment.

```
with open(cereal_csv,newline="") as csvfile:
```
There are a few things going on here:

First, our `with` statement is allowing us to catch `exceptions` in our code block without completely stopping our code from running. 

Second, our `open()` function is telling `python` to read the file at the address denoted by `cereal_csv`. `newline=''` involves a lower-level function of the reader and tells `python` how to handle different types of line separators - we don't need to worry about this very much right now.

Third, we are taking the object which we just created using the `open()` function and giving it the name `csvfile`. This object is an `iterable`, with each piece of that `iterable` being a `line` in the `cereal.csv` file.

```
  csvreader = csv.reader(csvfilem, deliiter=",")
```
We are creating a `reader` object which is an `iterable` based upon the contents of the opened `csvreader` object. We are also telling the code that the items in the `csv` are separated, or delimited, by - you guessed it - a comma! We'll talk a little more about what that `iterable` property means below:

```
  csv_header = next(csv_file)
```
It may be a little beyond the scope of this course to talk in-depth about *how* the `next()` function works, but essentially any object that is `iterable` does so because it has a roadmap that tells `python` how to get from one value to another. This is usually a straight line, from item 1, to item 2, etc. The `iterator` also has something like a `cursor`, which keeps track of where you're at in that list of items. `next()` basically returns the item that `cursor` is pointing at, then moves the `cursor` to wherever the roadmap tells it to go next. This is why `csv_header` contains the file's header, but `csvreader` now starts off on the line immediately following the header, since it's still connected to the `csvfile` object.

```
  print(f"header: {csv_header}")
```
This is a formatted print function which shows the items you just pulled from the header. 
```
  for row in csvreader:
```
Since `csvreader` is an `iterable`, we can use a `for loop` on it. This basically tells `python` to loop through every item (in this case, each row from our `cereal.csv`), and for each item it gets, do something with it. It temporarily gives the item the name `row`. Here, each `row` object is a `python` `list`, basically a series of values which can be `iterated` through or selected individually.

```
    if float(row[7]) >= 5:
```
Now that we have our `row` objects, everything in the `for loop` will be performed on one row from `csvreader` at a time. So let's talk about what this statement will do to an individual `row`:

First, we select `row[7]` - counterintuitively, the 8th item in the `row` object. This is because when we `index` - or select by position - in `python`, we do so starting with item number `0`. As we will see later, this is not the case in every language `R` `iterables` start at `1`. 

We need to convert the item from its current data type (a `string` of characters) to something which `python` can compare to other numbers. This means using some `numeric` data type, like `int` or `float`. 

Some of the items in the `fiber` column contain decimal values, like `1.5`. These can't be precisely represented as `integers`, since python may round the number or otherwise not represent it properly, so we'll need to make sure to have `python` `cast` - or convert - the value into a `float`. That's what `float()` does. 

Finally, we will use a comparison to determine whether the `float` we just retrieved is greater than or equal to `5`. Greater than or equal to is expressed as `>=` in `python`. When we're using an `if` statement, we want whatever's after it to be either `True` or `False`. This `>=` comparator will return one of these, and the code contained in the `if` statement will be executed if that value is `True`.

In [ ]:
#This command, while not covered above, is what measures the current time.
time1 = time.time()
#Define the path which will lead to the cereal.csv file.
cereal_csv = os.path.join("python_csv_writeup","Resources", "cereal.csv")
# Open and read csv
with open(cereal_csv, newline="") as csvfile:
  csvreader = csv.reader(csvfile, delimiter=",")
  # Read the header row first (skip this part if there is no header)
  csv_header = next(csvfile)
  print(f"Header: {csv_header}")
  # Read through each row of data after the header
  for row in csvreader:
    # Convert row to float and compare to grams of fiber
    if float(row[7]) >= 5:
      print(row)
time1 = time.time() - time1

print(f"\nTime: {time1}")

Header: ﻿name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
['100% Bran', 'N', 'C', '70', '4', '1', '130', '10', '5', '6', '280', '25', '3', '1', '0.33', '68.402973']
['All-Bran', 'K', 'C', '70', '4', '1', '260', '9', '7', '5', '320', '25', '3', '1', '0.33', '59.425505']
['All-Bran with Extra Fiber', 'K', 'C', '50', '4', '0', '140', '14', '8', '0', '330', '25', '3', '1', '0.5', '93.704912']
['Bran Flakes', 'P', 'C', '90', '3', '0', '210', '5', '13', '5', '190', '25', '3', '1', '0.67', '53.313813']
['Fruit & Fibre Dates; Walnuts; and Oats', 'P', 'C', '120', '3', '2', '160', '5', '12', '10', '200', '25', '3', '1.25', '0.67', '40.917047']
['Fruitful Bran', 'K', 'C', '120', '3', '0', '240', '5', '14', '12', '190', '25', '3', '1.33', '0.67', '41.015492']
['Post Nat. Raisin Bran', 'P', 'C', '120', '3', '1', '200', '6', '11', '14', '260', '25', '3', '1.33', '0.67', '37.840594']
['Raisin Bran', 'K', 'C', '120', '3', '1', '210', '5', '14', '12'

#Base `python` + List Comprehension

Something really cool about python is its modular ability to generate `lists` and `iterables`. We might have covered this briefly today, but I want to take the opportunity to show you how the above code could be converted into `list comprehension` notation. 

Basically, `list comprehension` is a way to construct `iterable` objects without using `for loops`. They are usually a little faster, since statements written in `list comprehension` are often a little faster for `python` than `for loops`, though they can often sacrifice readability. You may notice in the following example that


```
  matching_rows = [row
                   for row in csvreader 
                   if float(row[7]) >= 5
                   ]
  print(matching_rows)
```

is a bit harder to read at first glance than



```
  for row in csvreader:
    if float(row[7]) >= 5:
      print(row)
```

However, `list comprehension` has a lot of benefits, and it is widely utilized. [Here's](https://www.geeksforgeeks.org/python-list-comprehension-and-slicing/) a write-up if you reckon you'd like to learn more.

One more thing: the below code contains the following print statement:

```
  print(*matching_rows, sep='\n')
```

which may look kind of weird. Basically, the `*` in front of the `matching_rows` list indicates we'd like to pass each `element` from that list as an individual argument to the print statement (it's the same as if we passed in something like `print(row1,row2,row3,row4, etc)`, then the `sep='\n'` means we'd like to separate each argument with a newline. This makes the print output look just like the one in the `for loop` above.




In [ ]:
#This command, while not covered above, is what measures the current time.
time2 = time.time()
#Let's define the path again
cereal_csv = os.path.join("python_csv_writeup","Resources", "cereal.csv")
# Open and read csv
with open(cereal_csv, newline="") as csvfile:
  csvreader = csv.reader(csvfile, delimiter=",")
  # Read the header row first (skip this part if there is no header)
  csv_header = next(csvfile)
  print(f"Header: {csv_header}")
  # Read through each row of data after the header, this time using list comprehension.
  matching_rows = [row
                   for row in csvreader 
                   if float(row[7]) >= 5
                   ]
  print(*matching_rows, sep='\n')

time2 = time.time() - time2

print(f"\nTime: {time2}")

Header: ﻿name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
['100% Bran', 'N', 'C', '70', '4', '1', '130', '10', '5', '6', '280', '25', '3', '1', '0.33', '68.402973']
['All-Bran', 'K', 'C', '70', '4', '1', '260', '9', '7', '5', '320', '25', '3', '1', '0.33', '59.425505']
['All-Bran with Extra Fiber', 'K', 'C', '50', '4', '0', '140', '14', '8', '0', '330', '25', '3', '1', '0.5', '93.704912']
['Bran Flakes', 'P', 'C', '90', '3', '0', '210', '5', '13', '5', '190', '25', '3', '1', '0.67', '53.313813']
['Fruit & Fibre Dates; Walnuts; and Oats', 'P', 'C', '120', '3', '2', '160', '5', '12', '10', '200', '25', '3', '1.25', '0.67', '40.917047']
['Fruitful Bran', 'K', 'C', '120', '3', '0', '240', '5', '14', '12', '190', '25', '3', '1.33', '0.67', '41.015492']
['Post Nat. Raisin Bran', 'P', 'C', '120', '3', '1', '200', '6', '11', '14', '260', '25', '3', '1.33', '0.67', '37.840594']
['Raisin Bran', 'K', 'C', '120', '3', '1', '210', '5', '14', '12'

#`pandas`

Now, we're going to do the same selection task, using objects & methods from the `pandas` library. 

There are dozens of ways to solve this challenge in `pandas`, but I picked one which is simple, to the point, and quick, without sacrificing too much efficiency. 

```
cereal_data_frame = pd.read_csv(cereal_csv, sep=",", header="infer")
```

We have the same `path` set in our `cereal_csv` variable here, since we're accessing the same file, so the first time our code differs here is when we read our `csv` file. `pandas` does a lot of work for us here - I added the `sep` argument in manually even though commas are the default delimiter. It also `infer`s the header type by default, but if you would like to use the `cereal_bonus.csv` file here, you would only need to change that argument to `header=False`.

What we are making here, in fact, is a `pandas` `data_frame` object. As you're working with it, you might recognize its similarity to the native `python` `dict` object. 

Since `pandas` has already taken care of the header, we don't need to worry about using the `next()` function to pull it out of our data. We just print out the header stored in the `cereal_data_frame` object:
```
print(f"Header: {cereal_data_frame.columns.tolist()}")
```

Now, we only need one line to select *every* cereal in the `DataFrame` with over 5g of fiber: 
```
matching_rows = cereal_data_frame.iloc[:,7] >= 5
```
Basically, we're taking our `cereal_data_frame` object and getting its `iloc` attribute. This attribute is also called `index-location` and allows us to select specific `slices` from our `DataFrame` object. Here our slice has two components, with order `[` `rows`,`columns` `]`: The first - `:` - is a shorthand for `all indices`; we select every row in the `DataFrame`. The second - `7` - indicates we select the 8th column. When we put these two statements together, they mean "Select the 8th column from every row in `cereal_data_frame`.". Pandas then lets us directly determine which of these items are greater than or equal to 5 using the same `>= 5` statement as above. 

Note, if we were sure we had a header for our `DataFrame`, we could also have done this:

```
matching_rows = cereal_data_frame['fiber'] >= 5
```

What this statement returns is a `Series`, similar to a `list`, which contains either `True` or `False` for each row in `cereal_data_frame`, indicating whether the `fiber` column in this row contained a value greater than or equal to 5. Then all we need to do is select the rows from `cereal_data_frame` which are labeled `true`:

```
print(cereal_data_frame.loc[matching_rows])
```

Here we use the `.loc` attribute of `cereal_data_frame` to get its row locations corresponding to `True` in `matching_rows`. Then, we print them! 


In [ ]:
#This command, while not covered above, is what measures the current time.
time3 = time.time()
#we make an empty string here, to later be filled by the code.
cereal_csv = os.path.join("python_csv_writeup","Resources", "cereal.csv")
# Open and read csv. infer what type the header is.
cereal_data_frame = pd.read_csv(cereal_csv, sep=",", header="infer")
print(f"Header: {cereal_data_frame.columns.tolist()}")
# Read through each row of data after the header, this time using list comprehension.
matching_rows = cereal_data_frame.iloc[:,7] >= 5
print(cereal_data_frame.loc[matching_rows])

time3 = time.time() - time3

print(f"\nTime: {time3}")

Header: ['name', 'mfr', 'type', 'calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'shelf', 'weight', 'cups', 'rating']
                                      name mfr type  ...  weight  cups     rating
0                                100% Bran   N    C  ...    1.00  0.33  68.402973
2                                 All-Bran   K    C  ...    1.00  0.33  59.425505
3                All-Bran with Extra Fiber   K    C  ...    1.00  0.50  93.704912
9                              Bran Flakes   P    C  ...    1.00  0.67  53.313813
27  Fruit & Fibre Dates; Walnuts; and Oats   P    C  ...    1.25  0.67  40.917047
28                           Fruitful Bran   K    C  ...    1.33  0.67  41.015492
52                   Post Nat. Raisin Bran   P    C  ...    1.33  0.67  37.840594
58                             Raisin Bran   K    C  ...    1.33  0.75  39.259197

[8 rows x 16 columns]

Time: 0.08949995040893555


#`R` Language

So, we've done the same task three different ways using `python`. Now I'm going to be showing you how this can be translated into a different coding language, which we may cover a bit in this course: `R`.

Basically, `R` is a language somewhat similar to - but *much* older than - `python`. It's used primarily by researchers and statisticians, but many of its previous use-cases are now being covered by `python`. Nonetheless, this was my first data-oriented programming languate, and I'd be remiss if I didn't show you a bit of it:

I'm not going to do a line-by-line here like I did above, but I am going to give you a little overview of some of `r`'s quirks:

One thing you might notice here is the presence of `<-` characters where normally you'd expect to see `=`. These two do have different purposes, but they're also a bit beyond the scope of this course. Basically a good rule-of-thumb is: use `<-` when declaring variables (like `some_variable <- 5`) and `=` when assigning arguments in function calls (like `some_function(some_arg=5)`. 

Another difference is that when you're calling the `R` equivalent of `slices`, you don't have to use the `:` to specify `all`: you can just leave it blank!

In [ ]:
#you use %%R to tell the system that the next lines will be written in R language.
%%R
#Get the system time
time4 <- Sys.time()
#file.path() is the R equivalent of the os.path.join() function in python.
cereal_csv <- file.path("python_csv_writeup","Resources", "cereal.csv")
#read.csv is the R equivalent of the pd.read_csv() function above. 
#Note that booleans in R are ALL CAPS.
cereal_data_frame <- read.csv(file = cereal_csv, sep=",", header=TRUE)

#cat is short for "print concatenated"; basically concatenates a bunch of strings and prints them
cat("Headers: ",colnames(cereal_data_frame))

#select the elements in cereal_data_frame where values in the 8th column
#are greater than or equal to 5.
results <- cereal_data_frame[cereal_data_frame[,8] >= 5,]


#print.data.frame(results, justify="left")

#subtract the current system time from the system time measured earlier
#to get the time the code took to run.
time4 <- Sys.time() - time4
print(time4)

Headers:  name mfr type calories protein fat sodium fiber carbo sugars potass vitamins shelf weight cups rating

R[write to console]: Error in `[.data.frame`(cereal_data_frame[index], 8) : 
  undefined columns selected




Error in `[.data.frame`(cereal_data_frame[index], 8) : 
  undefined columns selected


#Conclusion
Well, that's it! I gave a few differnet ways to solve this problem, which I hope can help bring together some of the things we're covering right now. Python is such a versatile language and there's always more to learn, so feel free to reach out if you have any questions or would like some clarification regarding anything I've written here. 